In [1]:
import bluepy
import netsci.visualization as nsv
import numpy as np
import scipy
import netsci.metrics.motifs as nsm
import networkx
import matplotlib.pyplot as plt
import netsci.models.random as nsr
import glob, os
from pathlib import Path
import pandas as pd
import seaborn as sns
from scipy import sparse

In [2]:
circ=bluepy.Circuit('/gpfs/bbp.cscs.ch/project/proj112/circuits/CA1/20211110-BioM/CircuitConfig')

cylinders = [i for i in circ.cells.targets if i.startswith('cylinder') ]
cylinders.remove('cylinders')
cylinders

In [ ]:
data_dict = {}

In [60]:
data_dict

{'mosaic': <456378x456378 sparse matrix of type '<class 'numpy.bool_'>'
 	with 156661043 stored elements in Compressed Sparse Column format>,
 'cylinder300': <6213x6213 sparse matrix of type '<class 'numpy.bool_'>'
 	with 1236142 stored elements in Compressed Sparse Column format>}

In [48]:
cur_path = os.getcwd()
target = 'cylinder300'
cur_conn_path = os.path.join(cur_path,f'cylinders_analysis/{target}/data/connectivity.npz')
#data_dict['mosaic'] = np.array(1*scipy.sparse.load_npz(cur_conn_path).todense())

In [49]:
data_dict[target] = scipy.sparse.load_npz(cur_conn_path)
sparse_matrix = data_dict[target]

In [50]:
sparse_matrix

<6213x6213 sparse matrix of type '<class 'numpy.bool_'>'
	with 1236142 stored elements in Compressed Sparse Column format>

In [51]:
sparse_matrix.shape

(6213, 6213)

In [52]:
nonzero_vals = sparse_matrix.count_nonzero()

In [53]:
nonzero_vals

1236142

In [46]:
conn_prob_mosaic = nonzero_vals / (sparse_matrix.shape[0]*sparse_matrix.shape[0]-1)

In [38]:
conn_prob_mosaic

0.032023269227142416

In [39]:
from scipy.sparse import random

In [40]:
np.random.randint(low=100,high=105)

104

In [13]:
n = 50
p = 0.1
seed = 123
S = random(n,n,density=p,random_state=seed)
sparse.save_npz('test_sparse',S)

In [14]:
c300_conn_path = os.path.join(cur_path,'cylinders_analysis/cylinder300/data/connectivity.npz')
c300_sparse = scipy.sparse.load_npz(c300_conn_path)
conn_prob_cylinder300 = c300_sparse.count_nonzero() / (c300_sparse.shape[0]*(c300_sparse.shape[0]-1))

In [47]:
conn_prob_cylinder300 / conn_prob_mosaic

5396.554916969373

In [54]:
sparse_matrix_csr = sparse_matrix.tocsr()

In [55]:
reciprocal_matrix = sparse_matrix_csr.multiply(sparse_matrix_csr.transpose())

In [56]:
num_reciprocals_both_diagonal = reciprocal_matrix.count_nonzero()
num_reciprocals_both_diagonal

129792

In [57]:
reciprocality_ca1 = num_reciprocals_both_diagonal / (sparse_matrix.shape[0]*sparse_matrix.shape[0]-1)

In [58]:
reciprocality_ca1

0.003362367882920626

In [20]:
reciprocality_ca1

6.56651732667348e-05

In [61]:
# takes some time
motifs_dict = {}
for key,value in data_dict.items():
    if key=='mosaic':
        continue
    cur_motifs =  nsm.motifs(1*value.todense(), algorithm='louzoun')
    motifs_dict[key] = cur_motifs
    print(key, cur_motifs,sep=': ')

cylinder300: [       -1        -1        -1 162025739  90327698 248200864  26151493
  58563579  19939329   2211349   3976892   3382804   5457691   1721108
   1735092    206611]


In [ ]:
motifs_dict

In [ ]:
df = pd.read_csv('cylinder_motifs.csv',index_col=0).T
df

In [ ]:
df = pd.DataFrame(motifs_dict)
df.T

In [ ]:
df = df.T

In [ ]:
df.iloc[:,3:]

In [ ]:
df.describe()

In [ ]:
df2 = df.copy()
df2 = df2.iloc[:,3:]
df2 = np.log10(df2)
df2.columns = np.arange(1,14)
df2

In [ ]:
df3 = df.copy()
df3 = df3.iloc[:,3:]
df3 = np.divide(df3,1e8)
df3.columns = np.arange(1,14)
df3

In [ ]:
ax = df.plot.area(figsize=(10,10))
ax.set_xticklabels(np.arange(7))

In [ ]:
df.index

In [ ]:
fig, axes = plt.subplots()
# plot violin. 'Scenario' is according to x axis, 
# 'LMP' is y axis, data is your dataframe. ax - is axes instance
sns.violinplot(data=df2, ax = axes)
plt.xlabel('Motifs')
plt.ylabel('Log(Occurences)')
plt.title('Motifs distribution in cylinders (n=10)')

In [ ]:
fig, axes = plt.subplots()
# plot violin. 'Scenario' is according to x axis, 
# 'LMP' is y axis, data is your dataframe. ax - is axes instance
sns.violinplot(data=df3, ax = axes)
plt.xlabel('Motifs')
plt.ylabel('Occurences x 10^8')
plt.title('Motifs distribution in cylinders (n=10)')

# Random Graph (average of 10 cylinders)

In [ ]:
n = 6213 # num of cells in cylinder300
p = 0.015

In [ ]:
sparsities = []
num_nodes = []
num_edges = []
for cylinder,value in data_dict.items():
    num_edges.append(len(np.where(data_dict[cylinder]!=0)[0]))
    num_nodes.append(data_dict[cylinder].shape[0])
    sparsities.append(len(np.where(data_dict[cylinder]!=0)[0]) / data_dict[cylinder].size) # divide nonzeros in adj matrix to its size to get its sparsity

In [ ]:
n = int(np.mean(num_nodes))
p = np.mean(sparsities)
print(n,p)

In [ ]:
erdos_mean = nsr.erdos_renyi(n, p)
erdos_motifs_mean = nsm.motifs(erdos_mean, algorithm='louzoun')

In [ ]:
nsv.bar_motifs(erdos_motifs_mean,title=f'Random Graph with n={n}, p={np.round(p,3)}')

# For cylinder300

In [ ]:
n = 6213 # num of cells in cylinder300
#p = 0.015 # based on PCs only
p = len(np.where(data_dict['cylinder300'] == 1)[0]) / n**2
erdos = nsr.erdos_renyi(n, p)

In [ ]:
erdos_motifs = nsm.motifs(erdos, algorithm='louzoun')

In [ ]:
nsv.bar_motifs(erdos_motifs,title=f'ER Graph (cylinder300) with n={n}, p={np.round(p,3)}')

In [ ]:
ax0, ax1 = nsv.bar_motifs(erdos_motifs,title=f'Random Graph with n={n}, p={np.round(p,3)}')
ax0.set_ylim([0,4e8])

# Dense ER

In [ ]:
erdos_dense = nsr.erdos_renyi(n, 0.1)
erdos_motifs_dense = nsm.motifs(erdos_dense, algorithm='louzoun')

In [ ]:
nsv.bar_motifs(erdos_motifs_dense,title='Random Graph p=0.1')

# Z score

In [ ]:
data_dict['cylinder300'].shape[0]

First, we need to calculate motifs for ER and dd networks 1000 times. then use their mean and std of motif structures to normalize our motifs

In [ ]:
num_repetitions = 1000
motifs = []
n = data_dict['cylinder300'].shape[0] # num of cells in cylinder300 = 6213
p = len(np.where(data_dict['cylinder300'] == 1)[0]) / n**2 # p = 0.032
for rep in range(num_repetitions):
    cur_erdos = nsr.erdos_renyi(n, p)
    cur_motifs = nsm.motifs(cur_erdos, algorithm='louzoun')
    motifs.append(cur_motifs)
np.save('ER_motifs',motifs)

In [ ]:
df2 = pd.read_csv('motifs_exp_control.csv',index_col=0)
df2

In [ ]:
dd_motifs = np.array([206002585, 103002838, 103030367,   8864003,  12187251,  12178900,
         2955388,    358112,    601756,    301333,    300414,     44187,
             590])

In [ ]:
all_motifs_dd = np.load('10_motifs_dd.npy')
dd_mean = np.mean(all_motifs_dd,axis=0)[3:]
dd_std = np.std(all_motifs_dd,axis=0)[3:]

In [ ]:
z_dd = (df2.loc['cylinder300'][3:].values - dd_mean) / dd_std

In [ ]:
nsv.bar_motifs(np.hstack([-np.ones(3),z_dd]),title='Z-score based on DD-graph')

In [ ]:
from matplotlib import pyplot as plt, gridspec
from netsci.metrics.motifs import *

In [ ]:
from netsci.visualization.motifs import *

In [ ]:
order = range(len(z_dd))
x = 1 + np.arange(len(order))
plt.figure()
gs = gridspec.GridSpec(2, 1, height_ratios=[10, 3])
ax0 = plt.subplot(gs[0])
ax0.bar(x, z_dd[order])
_=plt.xticks(x)

ax1 = plt.subplot(gs[1], sharex=ax0)
plot_all_triads(range(3,16), ax=ax1, label=False)
plt.axis('off')

In [ ]:
nsv.bar_motifs(np.hstack([-np.ones(3),dd_mean]),title='Triplets in Distance Dependent Graph (n=10)')

In [ ]:
gs = gridspec.GridSpec(2, 1, height_ratios=[10, 3])
ax0 = plt.subplot(gs[0])
plt.plot(x,z_dd,'k')
_=plt.xticks(x)
plt.legend(['dd'],loc='best')
plt.ylabel('z-score')

# after plotting the data, format the labels
#current_values = plt.gca().get_yticks()
# using format string '{:.0f}' here but you can choose others
#plt.gca().set_yticklabels(['{:.0f}'.format(x) for x in current_values])
plt.ticklabel_format(axis="y", style="sci", scilimits=(0,0))


ax1 = plt.subplot(gs[1], sharex=ax0)
_=plt.xticks(np.arange(1,14))
plot_all_triads(range(3,16), ax=ax1, label=False)
plt.axis('off')



In [ ]:
motifs_ER = []
for i in range(100,1100,100):
    cur_file = np.load(f'ER_motifs_{i}.npy')
    motifs_ER.extend(cur_file)
motifs_ER = np.array(motifs_ER)

In [ ]:
mean_motifs_ER = np.mean(motifs_ER,axis=0)
std_motifs_ER = np.std(motifs_ER,axis=0)

In [ ]:
nsv.bar_motifs(mean_motifs_ER,title='ER Graph, n=6213, p=0.032 (N=1000)')

In [ ]:
z_ER = (df2.loc['cylinder300'][3:].values - mean_motifs_ER[3:]) / std_motifs_ER[3:]

In [ ]:
fig = plt.figure(figsize=(10,5))
gs = gridspec.GridSpec(2, 1, height_ratios=[10, 3],figure=fig)
ax0 = plt.subplot(gs[0])
plt.plot(x,z_dd,'k')
plt.plot(x,z_ER,'gray')

_=plt.xticks(x)
plt.legend(['dd','ER'],loc='best')
plt.ylabel('z-score')

# after plotting the data, format the labels
#current_values = plt.gca().get_yticks()
# using format string '{:.0f}' here but you can choose others
#plt.gca().set_yticklabels(['{:.0f}'.format(x) for x in current_values])
plt.ticklabel_format(axis="y", style="sci", scilimits=(0,0))


ax1 = plt.subplot(gs[1], sharex=ax0)
_=plt.xticks(np.arange(1,14))
plot_all_triads(range(3,16), ax=ax1, label=False)
plt.axis('off')
plt.savefig('CA1_ER_dd_triads_zscore.png')



# Reciprocal ER

In [ ]:
reciprocal_conns_list = []
for i in range(data_dict['cylinder300'].shape[0]):
    for j in range(data_dict['cylinder300'].shape[1]):
        if i != j:
            if data_dict['cylinder300'][i,j] == data_dict['cylinder300'][j,i]:
                if data_dict['cylinder300'][i,j] != 0:
                    reciprocal_conns_list.append([i,j])

In [ ]:
len(reciprocal_conns_list)

In [ ]:
data = {tuple(sorted(item)) for item in reciprocal_conns_list}

In [ ]:
len(data)

In [ ]:
p_reciprocal = len(reciprocal_conns_list) / data_dict['cylinder300'].shape[0]**2
p_reciprocal

In [ ]:
n = 6213 # num of cells in cylinder300
p = len(np.where(data_dict['cylinder300'] == 1)[0]) / n**2
p_reciprocal = len(reciprocal_conns_list) / data_dict['cylinder300'].shape[0]**2

ER_reciprocal = nsr.erdos_renyi_reciprocal(n, p,p_reciprocal)

In [ ]:
print(n,p,p_reciprocal)

In [ ]:
ER_reciprocal_motifs = nsm.motifs(ER_reciprocal, algorithm='louzoun')

In [ ]:
nsv.bar_motifs(ER_reciprocal_motifs,title=f'ER Reciprocal n:{n}, p={np.round(p,3)}, r={np.round(p_reciprocal,3)}')

In [ ]:
reciprocal_ER_motifs = np.load('ER_motifs_reciprocal.npy')

In [ ]:
mean_motifs_ER_recip = np.mean(reciprocal_ER_motifs,axis=0)
std_motifs_ER_recip = np.std(reciprocal_ER_motifs,axis=0)
nsv.bar_motifs(mean_motifs_ER_recip,title='ER Graph, n=6213, p=0.032 (N=1000)')

In [ ]:
z_ER_reciprocal = (df2.loc['cylinder300'][3:].values - mean_motifs_ER_recip[3:]) / std_motifs_ER_recip[3:]

In [ ]:
fig = plt.figure(figsize=(10,5))
gs = gridspec.GridSpec(2, 1, height_ratios=[10, 3],figure=fig)
ax0 = plt.subplot(gs[0])
plt.plot(x,z_dd,'k')
plt.plot(x,z_ER,'gray')
plt.plot(x,z_ER_reciprocal,'silver')


_=plt.xticks(x)
plt.legend(['dd','ER','ER_reciprocal'],loc='best')
plt.ylabel('z-score')
plt.title('Comparison of CA1 triad motifs with different random models (cylinder300)')
# after plotting the data, format the labels
#current_values = plt.gca().get_yticks()
# using format string '{:.0f}' here but you can choose others
#plt.gca().set_yticklabels(['{:.0f}'.format(x) for x in current_values])
plt.ticklabel_format(axis="y", style="sci", scilimits=(3,3))

ax1 = plt.subplot(gs[1], sharex=ax0)
_=plt.xticks(np.arange(1,14))
plot_all_triads(range(3,16), ax=ax1, label=False)
plt.axis('off')
plt.savefig('CA1_ER_dd_triads_zscore.png')



In [ ]:
def normalize(z):
    return z/np.sqrt(np.sum(np.square(z)))

In [ ]:
fig = plt.figure(figsize=(10,5))
gs = gridspec.GridSpec(2, 1, height_ratios=[10, 3],figure=fig)
ax0 = plt.subplot(gs[0])
plt.plot(x,normalize(z_dd),'k')
plt.plot(x,normalize(z_ER),'gray')
plt.plot(x,normalize(z_ER_reciprocal),'silver')


_=plt.xticks(x)
plt.legend(['dd','ER','ER_reciprocal'],loc='best')
plt.ylabel('Normalized z-score')
plt.title('(Normalized) Comparison of CA1 triad motifs with different random models (cylinder300)')
# after plotting the data, format the labels
#current_values = plt.gca().get_yticks()
# using format string '{:.0f}' here but you can choose others
#plt.gca().set_yticklabels(['{:.0f}'.format(x) for x in current_values])
#plt.ticklabel_format(axis="y", style="sci", scilimits=(0,0))

ax1 = plt.subplot(gs[1], sharex=ax0)
_=plt.xticks(np.arange(1,14))
plot_all_triads(range(3,16), ax=ax1, label=False)
plt.axis('off')
plt.savefig('CA1_ER_dd_triads_zscore_norm.png')

